In [1]:
from qiskit import *
from qiskit.algorithms.optimizers import SLSQP 
from qiskit.quantum_info import SparsePauliOp 
from qiskit.primitives import Estimator 
import numpy as np 
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.algorithms.minimum_eigen_solvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

from qiskit_nature import settings
settings.tensor_unwrapping = False
settings.use_pauli_sum_op = False

driver = PySCFDriver(atom='O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0', unit=DistanceUnit.ANGSTROM, charge=0, spin=0, basis='sto3g')

electronic_structure_problem = driver.run()

print('Electonic structue problem (Molecule) ')
print(electronic_structure_problem.molecule)
print('(Basis) = ', electronic_structure_problem.basis)
print('(num_spatial_orbitals) = ', electronic_structure_problem.num_spatial_orbitals)

/var/folders/n5/sr7wwb8n1wggd7jsk_k37l_w0000gn/T/ipykernel_13455/191729339.py:2: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.optimizers import SLSQP


Electonic structue problem (Molecule) 
Molecule:
	Multiplicity: 1
	Charge: 0
	Unit: Bohr
	Geometry:
		O	(0.0, 0.0, 0.0)
		H	(1.430522676295752, 1.1073795089951262, 0.0)
		H	(-1.430522676295752, 1.1073795089951262, 0.0)
	Masses:
		O	16
		H	1
		H	1
(Basis) =  ElectronicBasis.MO
(num_spatial_orbitals) =  7


/Users/venkat/opt/anaconda3/envs/QCML/lib/python3.9/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
mapper = JordanWignerMapper()
np_classical_solver = NumPyMinimumEigensolver()

/var/folders/n5/sr7wwb8n1wggd7jsk_k37l_w0000gn/T/ipykernel_13455/1989761901.py:2: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  np_classical_solver = NumPyMinimumEigensolver()


#### Reduce the dimension of the problem

In [3]:
# Activate space transformer
active_space_transformer = ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=3)
electronic_structure_problem = active_space_transformer.transform(electronic_structure_problem)


#### Calculate initial state to be provided to the QPE

In [4]:
initial_state=HartreeFock(
    electronic_structure_problem.num_spatial_orbitals,
    electronic_structure_problem.num_particles,
    mapper)

initial_state.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
          
q_2: ─────
     ┌───┐
q_3: ┤ X ├
     └───┘
q_4: ─────
          
q_5: ─────

In [5]:
from qiskit.algorithms import HamiltonianPhaseEstimation
from qiskit.primitives import Sampler
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver


FermionicOpWithCoefficients = electronic_structure_problem.hamiltonian.second_q_op()
print("Number of spin orbitals:", FermionicOpWithCoefficients.num_spin_orbitals, electronic_structure_problem.num_particles)

qubit_mapper = ParityMapper(num_particles=electronic_structure_problem.num_particles)
qubit_hamiltonian = mapper.map(FermionicOpWithCoefficients)


Number of spin orbitals: 6 (1, 1)


#### Classical solution

In [6]:
numpy_solver = NumPyMinimumEigensolver()
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
calc = GroundStateEigensolver(qubit_mapper, numpy_solver)

res = calc.solve(electronic_structure_problem)

print(res.eigenvalues.real)

[-1.66421877]


#### Quantum Phase estimation 

In [7]:
print('No of qubits required for the solution : ', qubit_hamiltonian.num_qubits)
sampler = Sampler()
pe = HamiltonianPhaseEstimation(num_evaluation_qubits=7, sampler=sampler)
result = pe.estimate(qubit_hamiltonian, initial_state)
print(result.most_likely_eigenvalue.real)

No of qubits required for the solution :  6
-1.6564150326191822
